In [ ]:
# Importing Jupyter Black Formatter.
import jupyter_black

jupyter_black.load()

# ICS 214 IT Workshop III (Python) | IIIT Kottayam
# Session 7 - Database Processing: Python and Databases | Tuesday, December 13, 2022
#### **Author:** Anmol Krishan Sachdeva (@greatdevaks)

## Databases

- Flat files? CSV, JSON, XML, etc.?
- SQLite and SQLAlchemy?
- MySQL and PostgreSQL?
- NoSQL?
- Caches?
- Queues?

### Data Storage with Flat Files: CSV

- Flat files are generally human-readable files for storing data.
- Data can be stored in CSV, JSON, XML, or custom formats.

In [ ]:
# Example: CSV Flat File Reading.

from collections import defaultdict
import csv

authors = defaultdict(int)
publishers = defaultdict(int)

with open("books.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        name = f"{row['first_name']} {row['last_name']}"
        authors[name] += 1
        publishers[row["publisher"]] += 1

print("#Books published by different Authors:")
for name in authors.keys():
    print(f"{name} => {authors[name]}")

print("\n")

print("#Books published by different Publishers:")
for name in publishers.keys():
    print(f"{name} => {publishers[name]}")

In [ ]:
# Example: CSV Flat File Writing.

import csv

with open("books.csv", "a", newline="\n") as csvfile:
    fieldnames = ["first_name", "last_name", "title", "publisher"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csvfile.write("\n")
    #     writer.writeheader()
    writer.writerow(
        {
            "first_name": "Baker",
            "last_name": "Fritz",
            "title": "Into the Universe",
            "publisher": "Galaxy",
        }
    )
    writer.writerow(
        {
            "first_name": "Roger",
            "last_name": "Djokovic",
            "title": "Story of Sun",
            "publisher": "WhiteHouse",
        }
    )

### SQLite

- No massive database servers needed.
- Available in Python as a built-in library.
- Data can be saved in a single file.
- Extremely light-weight database.
- Relational database characteristics.
- Not comparable to client/server SQL database engines like MySQL, PostgreSQL, etc.
- Scenarios:
    - Data Storage
    - Data Analysis
    - Data Transfer
    - Data persistance on small devices like IOT devices or Embedded decives
    - Temporary database
    - Light websites
    - Desktop applications
    - Local caching purposes
    - Local POCs
    - One write at a time with limited number of readers

In [ ]:
# Example: SQLite Database.

import sqlite3 as sl

con = sl.connect(
    "my-test.db"
)  # Establishing a Database Connection. If the database is not present, it is created.

In [ ]:
# Create a table `USER` with three columns `id`, `name`, and `age`.
with con:
    con.execute(
        """
        CREATE TABLE USER (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            age INTEGER
        );
    """
    )

In [ ]:
# Insert records into the `USER` table.
sql = "INSERT INTO USER (id, name, age) values(?, ?, ?)"
data = [(1, "Alice", 21), (2, "Bob", 22), (3, "Chris", 23)]

with con:
    con.executemany(sql, data)

In [ ]:
# Querying the `USER` table.abswith con:
data = con.execute("SELECT * FROM USER")
# data = con.execute("SELECT * FROM USER WHERE age <= 22")
for row in data:
    print(row)

#### Take Home Assignment

- Update records.
- Delete records.

### SQLAlchemy

- Third-party Python library.
- Offers ORM (Object Relational Mapping) capabilities.
- Provides methods and statements to interact with database.

In [ ]:
# Example: SQLAlchemy.

import sqlalchemy as sqldb

table_name = "USER"
db_engine = sqldb.create_engine("sqlite:///my-test.db")
db_connection = db_engine.connect()
metadata = sqldb.MetaData()
user_table = sqldb.Table(table_name, metadata, autoload=True, autoload_with=db_engine)

In [ ]:
# Print the table column names.
print(user_table.columns.keys())

In [ ]:
# Print table metadata.
print(repr(metadata.tables[table_name]))

In [ ]:
# Querying: Equivalent to 'SELECT * FROM USER'.
query = sqldb.select([user_table])
print(query)

In [ ]:
result = db_connection.execute(query).fetchall()
print(result)

In [ ]:
# Querying: Equivalent to 'SELECT * FROM USER WHERE age <= 22'.
query = sqldb.select([user_table]).where(user_table.columns.age <= 22)
print(query)

In [ ]:
result = db_connection.execute(query).fetchall()
print(result)

### Pickling

- Helps store state of objects into databases.
- The state can be exchanged over network as well.
- A concept of Serialization/Marshalling is used.
    - A way to convert and store a data structure into a linear format that can be exchanged over the network.
        - Think of having some data converted into byte streams.
- Deserialization/Unmarshalling is conversion of stream of bytes into the real data.
- Very common in Data Science and Neural Network processing.
- `pickle` module can be used.
    - Other common modules for Serialization/Deserialization are:
        - `json`
        - `marshal`
- Pickled objects can be further compressed.

In [ ]:
# Example: Pickling/Unpickling.

import pickle


class PickleExampleString:
    a_number = 35
    a_string = "hey"
    a_list = [1, 2, 3]
    a_dict = {"first": "a", "second": 2, "third": [1, 2, 3]}
    a_tuple = (22, 23)


class PickleExampleFile:
    a_number = 20
    a_string = "awesome"
    a_list = [1, 2, 3, 4]
    a_dict = {"first": "a", "second": 2, "third": [1, 2, 3, 4]}
    a_tuple = (22, 23)


my_object_string = PickleExampleString()
my_object_file = PickleExampleFile()

my_pickled_object = pickle.dumps(my_object_string)  # Pickling the object.
print(f"This is my pickled object:\n{my_pickled_object}\n")

my_object_string.a_dict = None

my_unpickled_object = pickle.loads(my_pickled_object)  # Unpickling the object.
print(f"This is a_dict of the unpickled object:\n{my_unpickled_object.a_dict}\n")
print(f"This is a_dict of the original object:\n{my_object_string.a_dict}\n")

with open("sample_dump.pickle", "wb") as picklefile:
    pickle.dump(my_object_file, picklefile)

with open("sample_dump.pickle", "rb") as picklefile:
    unserialized_data = pickle.load(picklefile)
print(f"This is a_dict of the unpickled file object:\n{unserialized_data.a_dict}\n")

### SQL vs. NoSQL

- SQL: Relational; NoSQL: Non-Relational
- SQL: Table-based; NoSQL: Document-based, key-value pair-based, graph, wide-column based, etc.
- SQL: Strict Schema; NoSQL: Schemaless
- SQL: SQL Query Language; NoSQL: Database dependent
- SQL: Scaling through replicas, sharding, or vertical scaling; NoSQL: Horizontal Scaling possible
- Examples:
    - SQL:
        - MySQL
        - PostgreSQL
        - MSSQL
    - NoSQL:
        - MongoDB
        - Aerospike
        - Cassandra
        - Redis
        
#### MongoDB

- Document-based NoSQL; JSON-like documents.
- Maintains `Collections` of `Documents` instead of `Tables` of `Rows`.

In [ ]:
.# Example: MongoDB.

import pprint
from pymongo import MongoClient

# client = MongoClient(host="localhost", port=27017)
client = MongoClient()

tutorial1 = {
    "title": "Working With JSON Data in Python",
    "author": "Daniel",
    "contributors": ["Max", "Dan", "John"],
    "url": "https://mock.com/python-json/",
}

# Establishing connectivity with the Database `tutorial`.
db = client.tutorial

# Writing data to a MongoDB database named `tutorial`.
tutorial = db.tutorial
result = tutorial.insert_one(tutorial1)

daniel_tutorial = tutorial.find_one({"author": "Daniel"})
client.close()

# Reading data from a MongoDB database named `tutorial`.
with MongoClient() as client:
    db = client.tutorial
    for doc in db.tutorial.find():
        pprint.pprint(doc)